# concordance

> Functionality for concordance analysis.

In [ ]:
#| default_exp concordance

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import time
import numpy as np
import polars as pl
import math
from fastcore.basics import patch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE, EOF_TOKEN_STR, ERR_TOKEN_STR, set_logger_state


In [ ]:
#| export
class Concordance:
	""" Class for concordancing. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| exporti
@patch
def _get_concordance_sort(self:Concordance, 
						 token_positions: list[np.ndarray], # token index to get sort columns for
						 sort_columns: list # sort columns to use
						 ) -> tuple[np.ndarray, np.ndarray]: # token ids for first sort column and corresponding sort order
	""" Get the first sort column for a concordance. """

	start_time = time.time()
	index = 'orth_index'
	seq = np.array(token_positions[0]+sort_columns[0])
	sort_column_ids = self.corpus.get_tokens_by_index('orth_index')[seq]
	sort_column_order = self.corpus.token_ids_to_sort_order(sort_column_ids)
	logger.info(f'Concordance sort column ({sort_column_ids.shape[0]}) retrieval time: {(time.time() - start_time):.5f} seconds')
	return sort_column_ids, sort_column_order


In [ ]:
#| hide
import os

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

path_to_toy_corpus = f'{save_path}toy.corpus'
path_to_brown_corpus = f'{save_path}brown.corpus'
path_to_reuters_corpus = f'{save_path}reuters.corpus'

In [ ]:
#| hide
toy = Corpus().load(path_to_toy_corpus)
report_toy = Concordance(toy)

In [ ]:
# load the corpus
brown = Corpus('brown').load(path_to_brown_corpus)

In [ ]:
# instantiate the Concordance class
report_brown = Concordance(brown)

In [ ]:
#| hide
token_str = 'dog'
token_sequence, index_id = brown.tokenize(token_str, simple_indexing=True)
token_positions = brown.get_token_positions(token_sequence, index_id)

sort_column_ids, sort_column_order = report_brown._get_concordance_sort(token_positions, [1, 2, 3])
print(sort_column_ids[:4])
print(brown.token_ids_to_tokens(sort_column_ids)[:4])
print(sort_column_order[:4])


[29064 38309 33838 15829]
['license' '.' 'owners' 'catchers']
[29512    40 36157  9356]


In [ ]:
#| export
@patch
def concordance(self: Concordance, 
				token_str: str, # token string to get concordance for 
				context_length:int = 5, # number of words to show on left and right of token string
				order:str='1R2R3R', # order of sort columns - one of 1L2L3L, 3L2L1L, 2L1L1R, 1L1R2R, 1R2R3R, LEFT, RIGHT
				page_size:int=PAGE_SIZE, # number of results to display per results page
				page_current:int=1, # current page of results
				show_all_columns:bool = False, # df with all columns or just essentials
				use_cache:bool = True # retrieve the results from cache if available
				) -> Result: # concordance report results
	""" Report concordance for a token string. """

	# DONE - reducing data retrieved to just the sort columns and then doing the concordance display separately here
	# DONE - speed up the sort so that does a partial sort (e.g. just one or two columns) to get position of the slice - then handle ordering with smaller slice of data
	# IDEA: potentially get sort columns until small enough result
	
	if order not in ['1L2L3L', '3L2L1L', '2L1L1R', '1L1R2R', '1R2R3R', 'LEFT', 'RIGHT']:
		raise ValueError(f'Invalid order: order must be one of: 1L2L3L, 3L2L1L, 2L1L1R, 1L1R2R, 1R2R3R, LEFT, RIGHT')
	
	if order == 'LEFT':
		order = '1L2L3L'
	elif order == 'RIGHT':
		order = '1R2R3R'

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)

	start_time = time.time()
	sequence_len = len(token_sequence[0])
	concordance_range = range(-1 * context_length, context_length + sequence_len)
	positional_columns = [str(x) for x in concordance_range]

	index = 'orth_index'

	cache_id = tuple(['concordance'] + list(token_sequence) + [order])

	if use_cache == True and cache_id in self.corpus.results_cache:
		logger.info('Using cached concordance results')
		positional_columns = self.corpus.results_cache[cache_id][0]
		concordance_df = self.corpus.results_cache[cache_id][1]
		total_count = self.corpus.results_cache[cache_id][2]
		total_docs = self.corpus.results_cache[cache_id][3]
		sort_columns = self.corpus.results_cache[cache_id][4]
	else:
		logger.info('Processing concordance results')
		token_positions = self.corpus.get_token_positions(token_sequence, index_id)

		if len(token_positions[0]) == 0:
			logger.info('No tokens found')
			return Result(type = 'concordance', df=pl.DataFrame(), title=f'Concordance for "{token_str}"', description=f'No matches', summary_data={}, formatted_data=[])

		if order == '1L2L3L':
			sort_columns = [-1,-2,-3]
		elif order == '3L2L1L':
			sort_columns = [-3,-2,-1]
		elif order == '2L1L1R':
			sort_columns = [-2,-1,sequence_len + 1 - 1]
		elif order == '1L1R2R':
			sort_columns = [-1,sequence_len + 1 - 1,sequence_len + 2 - 1]
		else:
			# i.e. 1R2R3R
			sort_columns = [sequence_len + 1 - 1,sequence_len + 2 - 1,sequence_len + 3 - 1]

		# getting first sort column here
		sort_column_ids, sort_column_order = self._get_concordance_sort(token_positions, sort_columns)
		
		concordance_df = pl.DataFrame([pl.Series(name='index', values=token_positions[0]), pl.Series(name='sort0', values=sort_column_order), pl.Series(name=str(sort_columns[0]), values=sort_column_ids)])
		concordance_df = concordance_df.sort('sort0')
		concordance_df = concordance_df.with_row_index('row')

		total_count = len(concordance_df)
		total_docs = len(np.unique(self.corpus.get_tokens_by_index('token2doc_index')[np.array(token_positions[0])])) # REFACTORED - was using old self.corpus.token2doc_index

		self.corpus.results_cache[cache_id] = [positional_columns, concordance_df, total_count, total_docs, sort_columns]

	# working out relevant slice to populate 
	resultset_start = page_size*(page_current-1)
	resultset_len = page_size
	resultset_end = min(resultset_start + resultset_len, len(concordance_df) - 1)
	
	start_order = concordance_df['sort0'][resultset_start]
	end_order = concordance_df['sort0'][resultset_end]
	start_order_pos = concordance_df.filter(pl.col("sort0") == start_order).head(1)['row'].item()
	end_order_pos = concordance_df.filter(pl.col("sort0") == end_order).tail(1)['row'].item()
	
	# populating a smaller chunk of the concordance report - as only need to retrieve/sort a subset
	concordance_result_df = concordance_df.slice(start_order_pos, end_order_pos - start_order_pos + 1)

	results_start_time = time.time()
	concordance_columns = []
	seq = concordance_result_df['index'].to_numpy()
	for pos in concordance_range:
		tokens = self.corpus.get_tokens_by_index(index)[np.array(seq+pos)] # REFACTORED - was using getattr call to get orth_index here
		concordance_columns.append(pl.Series(name=str(pos), values=tokens))
		if pos in sort_columns:
			column_name = 'sort'+str(sort_columns.index(pos))
			if column_name != 'sort0':
				concordance_columns.append(pl.Series(name=column_name, values=self.corpus.token_ids_to_sort_order(tokens)))
	logger.info(f'Concordance results ({len(concordance_columns[0])}) retrieval time: {(time.time() - results_start_time):.5f} seconds')

	concordance_result_df = concordance_result_df.with_columns(concordance_columns)
	#offsets_arr = np.array(self.corpus.offsets,dtype=np.uint64) # FIX
	#document_ids = np.searchsorted(offsets_arr, concordance_result_df['index'], side = 'right') - 1 
	document_ids = self.corpus.get_tokens_by_index('token2doc_index')[np.array(concordance_result_df['index'])] # REFACTORED to remove offsets functionality
	concordance_result_df = concordance_result_df.with_columns(pl.Series(name="document_id", values=document_ids))
	concordance_result_df = concordance_result_df.sort(['sort0','sort1','sort2'])
		
	# slicing this further to get only the required page of results and then populating with left, keyword, right strings
	concordance_view_df = concordance_result_df.slice(start_order_pos - resultset_start, page_size)

	concordance_left = []
	concordance_right = []
	concordance_keyword = []

	for pos in positional_columns:
		if int(pos) < 0:
			concordance_left.append(self.corpus.token_ids_to_tokens(concordance_view_df[str(pos)].to_numpy()))
		elif int(pos) == 0 or int(pos) < sequence_len:
			concordance_keyword.append(self.corpus.token_ids_to_tokens(concordance_view_df[str(pos)].to_numpy()))
		else:
			concordance_right.append(self.corpus.token_ids_to_tokens(concordance_view_df[str(pos)].to_numpy()))

	concordance_left = [(' '.join(column)).split(EOF_TOKEN_STR)[-1] for column in np.array(concordance_left).T]
	concordance_keyword = [' '.join(column) for column in np.array(concordance_keyword).T]
	concordance_right = [(' '.join(column)).split(EOF_TOKEN_STR)[0] for column in np.array(concordance_right).T]

	concordance_view_df = concordance_view_df.with_columns(pl.Series(name='left', values=concordance_left), pl.Series(name='node', values=concordance_keyword), pl.Series(name='right', values=concordance_right))

	total_pages = math.ceil(total_count/page_size)
	summary_data = {'total_count': total_count, 'total_docs': total_docs, 'page': page_current, 'total_pages': total_pages}
	formatted_data = [f'Total Concordance Lines: {total_count}', f'Total Documents: {total_docs}', f'Showing {min(page_size, total_count)} lines', f'Page {page_current} of {total_pages}']

	if show_all_columns == False:
		concordance_view_df = concordance_view_df[['document_id', 'left', 'node', 'right']]
	
	logger.info(f'Concordance report time: {(time.time() - start_time):.5f} seconds')

	return Result(type = 'concordance', df=concordance_view_df, title=f'Concordance for "{token_str}"', description=f'{self.corpus.name}, Context tokens: {context_length}, Order: {order}', summary_data=summary_data, formatted_data=formatted_data)


In [ ]:
#| hide
# no result
assert report_toy.concordance('dsahjhdsjhdsa', context_length=5).df.select(pl.len()).item() == 0

In [ ]:
#| hide
report_toy.concordance('the', context_length=5).display()

Concordance for "the" 
 
 
 Toy Corpus, Context tokens: 5, Order: 1R2R3R 
 
 
 Document Id 
 Left 
 Node 
 Right 
 
 
 
 
 5 
 
 The 
 cat is climbing a tree 
 
 
 3 
 
 The 
 cat is meowing . 
 
 
 1 
 
 The 
 cat sat on the mat 
 
 
 4 
 
 The 
 dog is barking . 
 
 
 6 
 
 The 
 dog is digging a hole 
 
 
 2 
 
 The 
 dog sat on the mat 
 
 
 1 
 The cat sat on 
 the 
 mat . 
 
 
 2 
 The dog sat on 
 the 
 mat . 
 
 
 
 
 
 Total Concordance Lines: 8 
 


 
 Total Documents: 6 
 


 
 Showing 8 lines 
 


 
 Page 1 of 1

In [ ]:
#| hide
reuters = Corpus().load(path_to_reuters_corpus)
conc_reuters = Concordance(reuters)

In [ ]:
#| hide
conc_reuters.concordance('the company said', context_length = 10, order='1R2R3R').display()

Concordance for "the company said" 
 
 
 Reuters Corpus, Context tokens: 10, Order: 1R2R3R 
 
 
 Document Id 
 Left 
 Node 
 Right 
 
 
 
 
 301 
 Pergamon 's expansion 
 programme and worldwide acquisition stategy . 
 The company said 
 
 that following these sales Pergamon 's ordinary shareholdings in 
 
 
 9,757 
 
 be paid in the transaction is grossly unfair . 
 The company said 
 
 the suit 's allegations are similar to those contained 
 
 
 8,330 
 
 fair treatment in the event of a takeover . 
 The company said 
 
 this is not in response to a known effort 
 
 
 3,298 
 Ron 
 McArthur , will also participate in ownership , 
 the company said 
 
 without elaborating . 
 The wholesale unit is a 
 
 
 2,744 
 acquire a controlling interest through a tender offer . 
 
 The company said 
 " The negotiations would determine the 
 terms under which 
 
 
 10,501 
 earnings by about 1.25 
 dlrs a share . 
 
 The company said 
 " this could bring earnings for 1987 below 
 Columbia 
 
 
 8,353 
 an average 0.185 ounce of gold per ton . 
 
 The company said 
 & lt;Manitoba Mineral Resources Ltd > owns the 
 remaining 
 
 
 2,186 
 end of last year . 
 In a statement , 
 the company said 
 , " The SEC action involves 
 events that occurred 
 
 
 8,898 
 lt;Nippon 
 Life Insurance Co > of Japan . 
 
 The company said 
 , " The discussions have led to a general 
 
 
 
 6,379 
 American Motors Corp. " 
 In a brief statement , 
 the company said 
 , " We are studying 
 the proposal . We 
 
 
 6,221 
 the establishment of additional special cost 
 escrow accounts , 
 the company said 
 , adding , that there may be no 
 cash 
 
 
 4,264 
 to be able to 
 close in near future , 
 the company said 
 , adding it is prepared 
 to close at any 
 
 
 6,319 
 177 mln dlrs after taxes . 
 In addition , 
 the company said 
 , Georgia Power 's contracts 
 with the joint owners 
 
 
 10,302 
 
 two mln dlrs against 1987 net income . 
 
 The company said 
 , however , that the sale will generate 
 substantial 
 
 
 4,664 
 
 results due to the conversion of debentures . 
 
 The company said 
 , however , it expects " positive " operating 
 
 
 
 6,464 
 . 
 As part of the distribution of assets , 
 the company said 
 , it 
 expects shareholders to receive an amount based 
 
 
 1,364 
 Commission . 
 Upon closing of the public offering , 
 the company said 
 , it 
 expects the secured institutional financing to increase 
 
 
 3,545 
 income . 
 While this is a preliminary estimate , 
 the company said 
 , it 
 may be used by shareholders when preparing 
 
 
 2,911 
 the merger . 
 As part of the transaction , 
 the company said 
 , it granted 
 IDC Acquisition an option to acquire 
 
 
 6,660 
 FDA last month . 
 While awaiting FDA approval , 
 the company said 
 , it is 
 proceeding with plans for commercialization through 
 
 
 
 
 
 Total Concordance Lines: 1092 
 


 
 Total Documents: 865 
 


 
 Showing 20 lines 
 


 
 Page 1 of 55

In [ ]:
report_brown.concordance('good at', context_length = 10, order='1R2R3R').display()

Concordance for "good at" 
 
 
 Brown Corpus, Context tokens: 10, Order: 1R2R3R 
 
 
 Document Id 
 Left 
 Node 
 Right 
 
 
 
 
 484 
 about twenty miles away , and he was also pretty 
 good at 
 anything in the carpentry line . was a vivid , 
 
 
 263 
 he says , ' as a storyteller and was precociously 
 good at 
 description , dialogue , and most of the other staples 
 
 
 479 
 and not a method of passing the day . was 
 good at 
 his job . probably was n't hard for him to 
 
 
 474 
 trying to flatter her vanity . You must have been 
 good at 
 history at school . did you go to school '' 
 
 
 82 
 enough of unequal merit , but all of them pretty 
 good at 
 that . consisted of a new arrangement of ` ` 
 
 
 474 
 Why not '' ? ? said . I 'm not 
 good at 
 that kind of thing '' . This afternoon let 's 
 
 
 
 
 
 Total Concordance Lines: 6 
 


 
 Total Documents: 5 
 


 
 Showing 6 lines 
 


 
 Page 1 of 1

In [ ]:
#| hide
congress = Corpus().load(f'{save_path}us-congressional-speeches-subset-100k.corpus')
report_congress = Concordance(congress)
congress._init_token_arrays()

In [ ]:
#| hide
%time report_congress.concordance('economy', context_length = 5, order='1R2R3R').display()

Concordance for "economy" 
 
 
 US Congressional Speeches Subset 100k, Context tokens: 5, Order: 1R2R3R 
 
 
 Document Id 
 Left 
 Node 
 Right 
 
 
 
 
 82,659 
 amounts that it throws our 
 economy 
 
 
 
 
 75,018 
 Honey . I shrun the 
 economy 
 " ? It is honest 
 
 
 19,176 
 getting away from " Coolidge 
 economy 
 " already . and making 
 
 
 6,729 
 . We are talking " 
 economy 
 " and at the same 
 
 
 83,170 
 further into an " innovating 
 economy 
 " based on a highly 
 
 
 66,298 
 the article " Approach to 
 Economy 
 " by Senator PROXMIRE in 
 
 
 45,693 
 and that the " new 
 economy 
 " does not need the 
 
 
 92,987 
 the State of the Black 
 Economy 
 " has some penetrating comments 
 
 
 76,840 
 this amendment ever yell " 
 economy 
 " in my ears again 
 
 
 15,941 
 concept of a " plutonium 
 economy 
 " is terrifying to say 
 
 
 74,726 
 series on " The American 
 Economy 
 " TV film were particularly 
 
 
 96,965 
 editorial entitled " Road to 
 Economy 
 " which appears in todays 
 
 
 63,855 
 entirely in the interest of 
 economy 
 . 
 
 
 
 54,296 
 of a more selfreliant Indian 
 economy 
 . 
 
 
 
 62,534 
 taxes levied upon the German 
 economy 
 . 
 
 
 
 96,146 
 administration to improve the agricultural 
 economy 
 . 
 
 
 
 23,959 
 is quite satisfied with the 
 economy 
 . 
 
 
 
 33,885 
 it is necessary for the 
 economy 
 . 
 
 
 
 11,798 
 It will be an ultimate 
 economy 
 . 
 
 
 
 11,866 
 of other areas of our 
 economy 
 . 
 
 
 
 
 
 
 Total Concordance Lines: 3758 
 


 
 Total Documents: 2684 
 


 
 Showing 20 lines 
 


 
 Page 1 of 188

CPU times: user 272 ms, sys: 201 ms, total: 473 ms
Wall time: 192 ms


In [ ]:
#| hide
congress.tokenize('government', simple_indexing=True)

([(np.int64(60063),)], 66)

In [ ]:
#| export
@patch
def concordance_plot(self: Concordance,
					 token_str: str,
					 page_size: int = 10):
	"""Display concordance plot."""

	import numpy as np
	import plotly.graph_objects as go
	from plotly.subplots import make_subplots
	import ipywidgets as widgets

	# Tokenize and get positions
	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)
	token_positions = self.corpus.get_token_positions(token_sequence, index_id)
	sequence_len = len(token_sequence[0])

	if len(token_positions[0]) == 0:
		print("No matches found.")
		return

	document_ids = self.corpus.get_tokens_by_index('token2doc_index')[token_positions[0]]
	unique_document_ids = np.unique(document_ids)
	num_docs = len(unique_document_ids)

	font_family = "'Segoe UI', Roboto, Oxygen, Ubuntu, Cantarell, 'Helvetica Neue', 'Fira Sans', 'Droid Sans', Arial, sans-serif;"
	plots_per_page = page_size
	num_pages = int(np.ceil(num_docs / plots_per_page))
	per_subplot_height = 50

	# Create FigureWidget with empty subplots
	fig = make_subplots(
		rows=plots_per_page,
		cols=1,
		vertical_spacing=0.05,
		subplot_titles=[None] * plots_per_page,
	)
	fig_widget = go.FigureWidget(fig)

	# Add background shapes and configure axes for each subplot
	for idx in range(plots_per_page):
		row = idx + 1
		fig_widget.add_shape(
			type="rect",
			x0=0, y0=0, x1=100, y1=1,
			line=dict(color="black", width=1),
			fillcolor="gray",
			opacity=0.08,
			layer="below",
			row=row, col=1
		)
		fig_widget.update_xaxes(range=[0, 100], visible=False, fixedrange=True, row=row, col=1)
		fig_widget.update_yaxes(range=[0, 1], visible=False, fixedrange=True, row=row, col=1)

	fig_widget.update_layout(
		height=per_subplot_height * plots_per_page,
		width=600,
		showlegend=False,
		title_text=f'Concordance Plot for "{token_str}"',
		title_x=0.5,
		margin=dict(t=40, b=20, l=80, r=20),
		font=dict(family=font_family, size=12),
	)

	# Slider for paging
	page_slider = widgets.IntSlider(value=1, min=1, max=num_pages, step=1, description='Page', layout=widgets.Layout(width='600px', margin='10px 0 10px 0'))

	footer = widgets.HTML(
		value=f"<div style='text-align: left; font-size: 12px; color: black; margin-left: 80px;margin-bottom:10px;line-height:1.7;'>{self.corpus.name}<br>Total Documents: {num_docs}<br>Total Concordance Lines: {len(token_positions[0])}</div>"
	)

	def update(change):
		page = change['new']
		fig_widget.data = ()  # Remove all previous traces
		fig_widget.layout.annotations = ()

		start = (page - 1) * plots_per_page
		end = min(start + plots_per_page, num_docs)
		page_document_ids = unique_document_ids[start:end]

		# Get min/max positions for normalization
		doc_range = self.corpus.tokens.with_row_index('position').filter(
			pl.col('token2doc_index').is_in(page_document_ids)
		).group_by('token2doc_index').agg([
			pl.col('position').min().alias('min'),
			pl.col('position').max().alias('max')
		]).collect()

		# Prepare normalized positions for each doc on this page
		documents = [[] for _ in range(len(page_document_ids))]
		normalized_documents = [[] for _ in range(len(page_document_ids))]
		examples = [[] for _ in range(len(page_document_ids))]
		for i, doc_id in enumerate(document_ids):
			if doc_id not in page_document_ids:
				continue
			doc_index = np.where(page_document_ids == doc_id)[0][0]
			documents[doc_index].append(token_positions[0][i])
			position_min, position_max = doc_range.filter(
				pl.col('token2doc_index') == doc_id
			).select(['min', 'max']).to_numpy()[0]
			norm_pos = (token_positions[0][i] - position_min) / (position_max - position_min) * 100 if position_max > position_min else 0
			normalized_documents[doc_index].append(norm_pos)
			tokens_for_example = self.corpus.get_tokens_by_index('orth_index')[token_positions[0][i]-5:token_positions[0][i]+6]
			
			if self.corpus.EOF_TOKEN in tokens_for_example:
				positions = np.where(tokens_for_example == self.corpus.EOF_TOKEN)[0]
				if len(positions) > 0 and positions[0] < 5:
					tokens_for_example = tokens_for_example[positions[0] + 1:]
				else:
					positions = np.where(tokens_for_example == self.corpus.EOF_TOKEN)[0]
					if len(positions) > 0 and positions[-1] > 5:
						tokens_for_example = tokens_for_example[:positions[-1]]
			tokens_for_example = self.corpus.token_ids_to_tokens(tokens_for_example)
			examples[doc_index].append(' '.join(tokens_for_example))

			# yref = "y" if i == 0 else f"y{i + 1} domain"

		n_plots_this_page = end - start
		if n_plots_this_page < plots_per_page or ('old' in change and change['old'] == num_pages):
			fig_widget.layout.shapes = ()
			for idx in range(n_plots_this_page):
				row = idx + 1
				fig_widget.add_shape(
					type="rect",
					x0=0, y0=0, x1=100, y1=1,
					line=dict(color="black", width=1),
					fillcolor="gray",
					opacity=0.08,
					layer="below",
					row=row, col=1
				)

		# Add traces for each subplot
		for idx, positions in enumerate(normalized_documents):
			row = idx + 1
			for pos, x0 in enumerate(positions):
				fig_widget.add_trace(
					go.Scatter(
						x=[x0, x0],
						y=[0, 1],
						mode='lines',
						line=dict(color='black', width=2),
						showlegend=False,
						hoverinfo='text',
						hovertext=f"{examples[idx][pos]}" if examples[idx] else ""
					),
					row=row, col=1
				)

			yref = "y" if row == 1 else f"y{row} domain"

			# doc id from document_ids
			doc_id = page_document_ids[idx]
			fig_widget.add_annotation(
				text=f"Doc {doc_id}",
				xref="paper", yref=yref,
				x=-0.03, y=0.65,
				showarrow=False,
				xanchor="right",
				yanchor="middle",
				font=dict(size=12, family = font_family)
			)

			lines_count = len(normalized_documents[idx])
			if lines_count == 0:
				lines_string = "No lines"
			elif lines_count == 1:
				lines_string = "1 line"
			else:
				# pluralize 'line' based on count
				# e.g. "5 lines"
				lines_string = f"{lines_count} lines"

			fig_widget.add_annotation(
				text=f"{lines_string}",
				xref="paper", yref=yref,
				x=-0.03, y=0.22, 
				showarrow=False,
				xanchor="right",
				yanchor="middle",
				font=dict(size=11, color="gray", family = font_family)
			)

	# Initial plot
	update({'new': 1})
	page_slider.observe(update, names='value')

	display(page_slider)
	display(fig_widget)
	display(footer)






In [ ]:
set_logger_state('verbose')
report_congress.concordance_plot('peace')

2025-06-08 20:23:39 - INFO - tokenize - Tokenization time: 0.00021 seconds
2025-06-08 20:23:39 - INFO - get_token_positions - Token indexing (3257) time: 0.01952 seconds


IntSlider(value=1, description='Page', layout=Layout(margin='10px 0 10px 0', width='600px'), max=192, min=1)

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'hovertext': 'attitude toward the achievement of peace and stability in this hemisphere',
              'line': {'color': 'black', 'width': 2},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'uid': '0d1b95a3-1848-4af7-a896-87a661facbfe',
              'x': [70.40650406504065, 70.40650406504065],
              'xaxis': 'x',
              'y': [0, 1],
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'hovertext': 'for improved world relations and peace is so great that certainly',
              'line': {'color': 'black', 'width': 2},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'ddedf42e-0b29-4e35-aacc-b4164edc35bd',
              'x': [54.97076023391813, 54.97076023391813],
              'xaxis': 'x2',
              'y': [0, 1],
              'yaxis': 'y2'},

HTML(value="<div style='text-align: left; font-size: 12px; color: black; margin-left: 80px;margin-bottom:10px;…

In [ ]:
#| hide
# congress.text(125).display()

In [ ]:
#| hide
# congress.tokens.with_row_index('position').filter(pl.col('lower_index') == 60063).head(21).collect()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()